In [19]:
import numpy as np
from IPython.display import clear_output
from tensorflow.keras.models import load_model

def display_board(board):
    """
    Displays the Connect 4 board from a 6x7x2 representation.
    Uses 'X' for player +1 and 'O' for player -1.
    """
    clear_output()
    
    horizontal_line = '-' * (7 * 5 + 8)
    blank_line = '|' + ' ' * 5
    blank_line *= 7
    blank_line += '|'
    
    print('   0     1     2     3     4     5     6')
    print(horizontal_line)
    
    for row in range(6):
        print(blank_line)
        this_line = '|'
        for col in range(7):
            if board[row, col, 0] == 1:  # Player +1
                this_line += '  X  |'
            elif board[row, col, 1] == 1:  # Player -1
                this_line += '  O  |'
            else:  # Empty space
                this_line += '     |'
        print(this_line)
        print(blank_line)
        print(horizontal_line)
    
    print('   0     1     2     3     4     5     6')

In [20]:
def update_board(board_temp, color, column):
    """
    Updates the board (6x7x2) by placing a checker in the specified column.
    
    Parameters:
    - board_temp: 6x7x2 NumPy array representing the board
    - color: 'plus' for +1, 'minus' for -1
    - column: Integer (0-6) representing the column where the piece is dropped
    
    Returns:
    - Updated 6x7x2 board with the new piece added
    """
    board = board_temp.copy()
    
    # Find the lowest available row in the given column
    for row in range(5, -1, -1):  # Start from bottom row
        if board[row, column, 0] == 0 and board[row, column, 1] == 0:  # Check if empty
            if color == 'plus':
                board[row, column, 0] = 1  # Set +1 in first channel
                board[row, column, 1] = 0
            else:
                board[row, column, 0] = 0
                board[row, column, 1] = 1  # Set -1 in second channel
            return board  # Return updated board
    
    # If column is full, return the board unchanged
    return board

In [22]:
def check_for_win(board, col):
    """
    Checks for a win in the 6x7x2 board.

    Parameters:
    - board: 6x7x2 NumPy array
    - col: Integer (0-6) indicating the last column where a checker was dropped

    Returns:
    - 'v-plus', 'v-minus' for vertical win
    - 'h-plus', 'h-minus' for horizontal win
    - 'd-plus', 'd-minus' for diagonal win
    - 'nobody' if no win
    """
    nrow, ncol = 6, 7
    
    # Find the row of the last played move
    for row in range(6):
        if board[row, col, 0] == 1 or board[row, col, 1] == 1:
            break  # Found the last placed checker
    
    # Identify which player made the move
    player = "plus" if board[row, col, 0] == 1 else "minus"
    check_channel = 0 if player == "plus" else 1
    
    def check_direction(delta_row, delta_col):
        """Counts consecutive checkers in a specific direction"""
        count = 0
        r, c = row, col
        while 0 <= r < nrow and 0 <= c < ncol and board[r, c, check_channel] == 1:
            count += 1
            r += delta_row
            c += delta_col
        return count

    # **Check vertical (↓)**
    if check_direction(1, 0) >= 4:
        return f'v-{player}'
    
    # **Check horizontal (← →)**
    left_count = check_direction(0, -1)  # Count leftwards
    right_count = check_direction(0, 1)  # Count rightwards
    if left_count + right_count - 1 >= 4:
        return f'h-{player}'

    # **Check diagonal (↘ ↖)**
    down_right = check_direction(1, 1)
    up_left = check_direction(-1, -1)
    if down_right + up_left - 1 >= 4:
        return f'd-{player}'

    # **Check diagonal (↙ ↗)**
    down_left = check_direction(1, -1)
    up_right = check_direction(-1, 1)
    if down_left + up_right - 1 >= 4:
        return f'd-{player}'

    return 'nobody'  # No win detected

In [23]:
def swap_board(board):
    """
    Swap [0,1] to [1,0] and [1,0] to [0,1] in a given board only if player == -1.
    """
    board = np.array(board)  # Ensure it's a NumPy array

    # Create masks
    mask_01 = (board[:,:,0] == 0) & (board[:,:,1] == 1)  # Find [0,1]
    mask_10 = (board[:,:,0] == 1) & (board[:,:,1] == 0)  # Find [1,0]

    # Swap values
    board[mask_01] = [1, 0]
    board[mask_10] = [0, 1]

    return board  # Return the modified or original board 

In [24]:
class PositionalIndex(tf.keras.layers.Layer):
    def call(self, x):
        bs = tf.shape(x)[0] # extract batch size
        number_of_vectors = tf.shape(x)[1] # how many vectors - we know it should be m*n, but let's count to be sure...
        indices = tf.range(number_of_vectors) # index for each vector
        indices = tf.expand_dims(indices, 0) # reshape it appropriately
        return tf.tile(indices, [bs, 1]) # repeat for each batch

In [25]:
class ClassTokenIndex(tf.keras.layers.Layer):
    def call(self, x):
        bs = tf.shape(x)[0] # extract batch size
        number_of_vectors = 1 # how many vectors - we just want 1 (which is @ 0) ... we only want to generate 1 vector for the class token
        # now just get it to be the right size
        indices = tf.range(number_of_vectors) # index for each vector
        indices = tf.expand_dims(indices, 0) # reshape it appropriately
        return tf.tile(indices, [bs, 1]) # repeat for each batch

In [26]:
# Load the Transformer model
from tensorflow.keras.models import load_model

# cnn_model = load_model('cnn_model.h5')  # Commenting out CNN model
transformer_model = load_model('transformer_model.h5')  # Using Transformer model instead

ValueError: Unknown layer: 'PositionalIndex'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
def play_game():
    # Initialize an empty board
    board = np.zeros((6, 7, 2), dtype=int)

    # Ask the user if they want to go first
    user_first = input("Do you want to go first? (y/n): ").strip().lower()
    user_player = 'minus' if user_first == 'y' else 'plus'

    # Initialize the winner
    winner = 'nobody'

    # Display the initial empty board
    display_board(board)

    while winner == 'nobody':
        if user_player == 'minus':
            # User's turn
            move = int(input('Pick a move (0-6) for player minus: '))
            board = update_board(board, 'minus', move)
            display_board(board)
            winner = check_for_win(board, move)
            user_player = 'plus'  # Switch to AI's turn
        else:
            # AI's turn (Transformer model)
            swapped_board = swap_board(board)
            transformer_prediction = transformer_model.predict(swapped_board[np.newaxis, ...])
            transformer_move = np.argmax(transformer_prediction)
            board = update_board(board, 'plus', transformer_move)
            display_board(board)
            winner = check_for_win(board, transformer_move)
            user_player = 'minus'  # Switch to user's turn

    if winner in ['v-plus', 'h-plus', 'd-plus']:
        print('The winner is the Transformer AI!')
    else:
        print('The winner is YOU!!!!')

In [ ]:
play_game()